# Importar datos

In [13]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
import pickle

In [3]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

In [3]:
data = pd.read_csv('tweets_2022_abril_junio.csv')

In [4]:
data.head()

,id,created_at,screen_name,text,favorite_count,retweet_count
0,1512186166438637582,2022-04-07 21:50:51 UTC,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...,0,0
1,1512186202367045642,2022-04-07 21:51:00 UTC,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...,0,0
2,1512186287284924418,2022-04-07 21:51:20 UTC,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...,0,0
3,1512186335754301446,2022-04-07 21:51:32 UTC,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...,0,0
4,1512186407841767424,2022-04-07 21:51:49 UTC,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...,0,0


# FIlter inicial

Quiza filtar datos que no importan, como el created_at, o el favorite_count, retweet_count

In [5]:
filtered_data_raw = data.filter(items=['id', 'screen_name', 'text'])

In [6]:
filtered_data = filtered_data_raw.astype({'screen_name': 'string', 'text': 'string'})

In [7]:
filtered_data.dtypes

id                      int64
screen_name    string[python]
text           string[python]
dtype: object

Tambien, vemos que existen algunos ids duplicados, por lo que tambien los removemos

In [8]:
filtered_data.drop_duplicates(subset=['id'])

,id,screen_name,text
0,1512186166438637582,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,1512186202367045642,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,1512186287284924418,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,1512186335754301446,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,1512186407841767424,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...
...,...,...,...
4594975,1526652300709679104,Alebarrera74,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...
4594976,1526641118460334080,gigita29bq,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...
4594977,1526738292011462657,Elizabe81480339,RT @Gonz1Gorjeperez: @tere_marinovic https://t...
4594978,1526855280151056386,CastilloNafla,RT @Gonz1Gorjeperez: @tere_marinovic https://t...


In [9]:
filtered_data.to_csv('filtered_tweets_2022_abril_junio.csv', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

# Primero

Intente hacer un doble for en el dataframe, pero 1 iteracion tomaba como 1 seg. Entonces, el doble for iba a tomar la cantidad de filas * 1 seg, o sea, ~4500000 segundos = 52 dias. Obviamente no da.

# Crear shingles de los textos

In [3]:
filtered_data = pd.read_csv('filtered_tweets_2022_abril_junio.csv', index_col=0, quotechar='"')

## Ver despues si vale la pena pasarlo a lista, pa que la iteracion sea mas rapida (parece que usa mas ram si, pero como 1.1GB, igual es manejable).
# filtered_data = pd.read_csv('filtered_tweets_2022_abril_junio.csv', index_col=0, quotechar='"').values.tolist()
## Creo que no vale la pena, la diferencia es muy chica.

In [5]:
def k_shingle(text, k=5):
    shingles = set()
    for idx in range(len(text) - k + 1):
        curr_slice = text[idx : idx + k]
        shingles.add(curr_slice)
    return shingles

def k_shingle_update(text, k=5):
    shingles = set()
    shingles.update([text[idx : idx + k] for idx in range(len(text) - k + 1)])
    return shingles

k = 5

Una vez definido el k de los shingles, procedemos a borrar todos los tweets con menos de k caracteres, ya que a estos no le podemos obtener los shingles.

In [80]:
reduced_data = filtered_data[filtered_data['text'].apply(len) >= k]

Luego, nos damos cuenta que existen muchos tweets repetidos. Entonces, para evitar tratar con textos duplicados, guardamos una asociacion entre el texto, y el ID de los tweets que tienen ese texto

In [81]:
tweets = {}

for row in reduced_data.itertuples():
    try:
        tweets[row.text].append(row.id)
    except KeyError:
        tweets[row.text] = [row.id]

print(len(tweets))

1546393


| k | tweets  | 
|---|---------|
| 5 | 1548053 |
| 6 | 1547594 |
| 7 | 1547038 |
| 8 | 1546393 |

Vemos que existen solo ~1500000 de tweets unicos, o sea, un tercio de los originales. Entonces, tenemos que procesar un tercio de los documentos.

In [82]:
unique_tweets = list(tweets.keys())

In [83]:
import pickle

with open(f"unique_tweets_k={k}.pkl", 'wb') as f:
    pickle.dump(unique_tweets, f)

In [84]:
total_size(unique_tweets)

406427991

# Intentar sacar todos los shingles de a una

In [32]:
tweet_shingles = {}

# i = 0
# for row in filtered_data.itertuples():
#     if i % 100000 == 0:
#         print(f"i: {i}")
#     tweet_shingles[row.id] = k_shingle(row.text, k)
#     i += 1

i: 100000


In [5]:
import sys

In [24]:
list(tweet_shingles.keys())

[1512186166438637582,
 1512186202367045642,
 1512186287284924418,
 1512186335754301446,
 1512186407841767424,
 1512186484731744256,
 1512186668387913732,
 1512186805860380673,
 1512186831277895684,
 1512186985850544129,
 1512187050015203329,
 1512187110593376263,
 1512187189974683661,
 1512187226398076929,
 1512187244248936452,
 1512187273625972744,
 1512187297361502213,
 1512187344404639744,
 1512187478106468353,
 1512187489175281664,
 1512187512068112384,
 1512187536503816194,
 1512187776128757761,
 1512187813080444939,
 1512187839257235457,
 1512187845913550849,
 1512187864142217216,
 1512187940155273229,
 1512187943137415168,
 1512188012217655297,
 1512188044887035906,
 1512188055788027904,
 1512188071055433730,
 1512188081595904002,
 1512188253377507337,
 1512188352434384912,
 1512188356658049029,
 1512188426954559491,
 1512188437293543431,
 1512188538598694919,
 1512188769985773572,
 1512188948877090816,
 1512189064740540426,
 1512189069824086024,
 1512189102862794754,
 151218912

In [33]:
sys.getsizeof(tweet_shingles)

5242968

In [34]:
sys.getsizeof(tweet_shingles[1512187776128757761])

8408

In [35]:
len(tweet_shingles)

99990

In [31]:
len(tweet_shingles) * 8408

840715920

# Problemas con Shingles
Intente crear todos los shingles y guardarlos, pero me quede sin RAM (la mitad de los shingles usaba mas o menos 12GB de RAM). Posible solucion: Calcular los Shingles on-the-fly, al usar la funcion de hash, y guardar solo el ID de los textos parecidos. Cada shingle usa como 8.4KB, si lo implementamos con un set, con k=5. 

Para solucionar esto, calculamos primero todos los shingles existentes. Luego, generamos algun ordenamiento (permutacion) de estos shingles. Despues, para cada texto, computamos sus shingles, vemos el shingle que antes aparece segun esa permutacion, y ese es el valor de LSH(text). Entonces, guardamos solamente {bucket: text_id}, y no tenemos que guardar todos los shingles simultaneamente en memoria.

In [85]:
all_shingles = set()

i = 0
for row in tweets.keys():
    if i % 100000 == 0:
        print(f"i: {i}")
    current_shingles = k_shingle(row, k)
    all_shingles.update(current_shingles)
    i += 1

i: 0
i: 100000
i: 200000
i: 300000
i: 400000
i: 500000
i: 600000
i: 700000
i: 800000
i: 900000
i: 1000000
i: 1100000
i: 1200000
i: 1300000
i: 1400000
i: 1500000


In [86]:
len(all_shingles)

16068608

In [88]:
import random
import time

In [90]:
shingles_list = list(all_shingles)

In [91]:
shingles_dict = {shingle: idx for idx, shingle in enumerate(shingles_list)}

In [5]:
def generate_random_permutation(shingles_list):
    shuffled = random.sample(shingles_list, k=len(shingles_list))
    return {shingle: idx for idx, shingle in enumerate(shuffled)}

Ahora que tenemos todos los shingles y tenemos una funcion de permutacion, computamos el LSH de cada texto.

In [6]:
def compute_text_LSH(text, shingle_permutation):
    text_shingles = k_shingle(text, k)
    min_id = float('inf')
    for shingle in text_shingles:
        shingle_id = shingle_permutation[shingle]
        if shingle_id < min_id:
            min_id = shingle_id
    return min_id

def compute_data_LSH(data, shingle_permutation):
    ## Por ahora, data es el dataframe. Quiza despues cambiarlo por una lista, pa que la iteracion sea mas rapida
    ## Despues de intentarlo con lista, el improvement es como 1 segundo, y se demora 2 minutos, no hay mucha
    ## gracia de usar lista.

    ## En el libro tambien hay un capitulo pa obtener estos valores, quiza ver si es mejor que esto.
    locally_sensitive_hash = np.zeros(len(data), dtype=np.int32)
    i = 0
    for row in data:
        if i % 100000 == 0:
            print(f"i: {i}")
        # try:
        text_lsh = compute_text_LSH(row, shingle_permutation)
        locally_sensitive_hash[i] = text_lsh
        # except:
        #     print(f"Error i: {i}")
        #     print(row.text)
        #     print(len(row.text))
        #     raise TypeError
        i += 1
    return locally_sensitive_hash


def get_data_LSH_signature(data, hash_funcs=10):
    LSH_signature = np.zeros((hash_funcs, len(data)), dtype=np.int32)

    for row_idx in range(hash_funcs):
        random_perm = generate_random_permutation(shingles_list)
        row = compute_data_LSH(data, random_perm)
        LSH_signature[row_idx, :] = row
    
    return LSH_signature
    


# def compute_data_LSH_list(data, shingle_permutation):
#     ## Aqui, lo mismo de arriba, pero data es una lista
#     ## El elemento 0 es el ID, el 1 es el autor, el 2 es el texto
#     locally_sensitive_hash = {}
#     i = 0
#     for row in data:
#         if i % 100000 == 0:
#             print(f"i: {i}")
#         text_lsh = compute_text_LSH(row[2], shingle_permutation)
#         try:
#             locally_sensitive_hash[text_lsh].append(row[0])
#         except KeyError:
#             locally_sensitive_hash[text_lsh] = [row[0]]
#         i += 1
#     return locally_sensitive_hash

## Forma de definir funciones de hash adaptada de la actividad 6 del curso
## (https://github.com/IIC2440/Syllabus-2023-1/blob/main/Actividades/06%20-%20LocallySensitiveHashing/Shingling_jaccard_universalhashes.ipynb)
def create_hash(n):
    ## Primo de 9 digitos elegido al azar con https://bigprimes.org/
    p = 791639819
    a = random.randint(1, p - 1)
    b = random.randint(1, p - 1)
    return lambda x: ((a * x + b) % p) % n

## Funcion basada en hashing en vez de generar una permutacion.
## Deberia usar mucha menos memoria que la de arriba, pero puede tener
## colisiones.
def compute_LSH_signatures(data, hash_funcs=10):
    hashes = [create_hash(len(data)) for _ in range(hash_funcs)]
    LSH_signature = np.zeros((hash_funcs, len(data)), dtype=np.int32)

    i = 0
    # shingle_to_idx_time  = 0.000000001
    # hash_idx_time        = 0.000000001
    for row in tqdm(data):
        # if i % 100000 == 0:
        #     print(f"i: {i}")
        #     print(f"hash_idx time        : {hash_idx_time:3f} ({hash_idx_time / (hash_idx_time + shingle_to_idx_time)})")
        #     print(f"shingle_to_idx time  : {shingle_to_idx_time:3f} ({shingle_to_idx_time / (hash_idx_time + shingle_to_idx_time)})")
        row_shingles = list(k_shingle(row, k))
        

        # st = time.time()
        row_shingles_idx = [shingles_dict[shingle] for shingle in row_shingles]
        # end = time.time()
        # shingle_to_idx_time += end - st

        # st = time.time()
        row_lsh = [min(map(hash_func, row_shingles_idx)) for hash_func in hashes]
        # end = time.time()

        # hash_idx_time += end - st

        LSH_signature[:, i] = row_lsh
        i += 1
    
    return LSH_signature

Cada lsh abajo usa como 200MB. Ver si lo podemos hacer on-the-fly, sin guardar todas las tablas, sino evaluar los AND y OR de forma inteligente.

In [ ]:
lsh_signature = get_data_LSH_signature(list(tweets.keys()), 2)


Demoro 1m41s

In [8]:
hash_funcs = 200

hash_lsh_signature = compute_LSH_signatures(unique_tweets, hash_funcs)

In [96]:
np.save(f"hash_signatures_k={k}_hash_funcs={hash_funcs}.npy", hash_lsh_signature)

In [60]:
k = 6

In [61]:
hash_lsh_signature = np.load(f"hash_signatures_k={k}_hash_funcs={hash_funcs}.npy")

# LSH

Una vez que computamos las signatures, procedemos a hacer LSH. Para esto, primero definimos b y r, tal que b * r ~ 200. Luego, hasheamos los vectores de cada banda, y los que caigan en el mismo buckes seran los similares

In [62]:
r = 10
b = 20

similar_items = set()

for band in tqdm(range(b)):
    band_hash = {}
    for tweet in range(hash_lsh_signature.shape[1]):
        tweet_vector = tuple(hash_lsh_signature[band * r:(band + 1)*r, tweet])
        try:
            band_hash[tweet_vector].append(tweet)
        except KeyError:
            band_hash[tweet_vector] = [tweet]
    for key in band_hash:
        for item1 in band_hash[key]:
            for item2 in band_hash[key]:
                if item1 != item2:
                    ## Put bigger item first always, to remove
                    ## duplicates (if tuple (a, b) is in set, then
                    ## (b, a) wont be).
                    if item1 > item2:
                        similar_items.add((item1, item2))
                    else:
                        similar_items.add((item2, item1))
        

100%|██████████| 20/20 [01:53<00:00,  5.70s/it]


In [63]:
with open(f"similar_items_k={k}_hash_funcs={hash_funcs}_b={b}_r={r}.pkl", 'wb') as f:
    pickle.dump(similar_items, f)

In [64]:
len(similar_items)

3334975

In [58]:
getsizeof(similar_items)

134217944

In [ ]:
similar_tuples = list(similar_items)

similar_tup_idx = 85683
similar_tup = similar_tuples[similar_tup_idx]

print(unique_tweets[similar_tup[0]])
print(unique_tweets[similar_tup[1]])

# Otros tests

In [118]:
all_shingles_test = {}

k = 5

i = 0
for row in reduced_data.itertuples():
    if i % 100000 == 0:
        print(f"i: {i}")
    current_shingles = k_shingle(row.text, k)
    for shingle in current_shingles:
        try:
            all_shingles_test[shingle].append(row.id)
        except KeyError:
            all_shingles_test[shingle] = [row.id]
    i += 1

i: 0
i: 100000
i: 200000
i: 300000
i: 400000
i: 500000
i: 600000
i: 700000
i: 800000
i: 900000
i: 1000000
i: 1100000
i: 1200000
i: 1300000
i: 1400000
i: 1500000
i: 1600000
i: 1700000
i: 1800000
i: 1900000
i: 2000000
i: 2100000
i: 2200000
i: 2300000
i: 2400000
i: 2500000
i: 2600000
i: 2700000
i: 2800000
i: 2900000
i: 3000000
i: 3100000
i: 3200000
i: 3300000
i: 3400000
i: 3500000
i: 3600000
i: 3700000
i: 3800000
i: 3900000
i: 4000000
i: 4100000
i: 4200000
i: 4300000
i: 4400000
i: 4500000


In [111]:
all_shingles_test2 = {}
shingles_list_test = []

k = 5

i = 1
shingle_id = 0
for row in reduced_data.itertuples():
    if i % 100000 == 0:
        print(f"i: {i}")
    current_shingles = k_shingle(row.text, k)
    for shingle in current_shingles:
        try:
            curr_shingle_id = all_shingles_test2[shingle]
            shingles_list_test[curr_shingle_id].append(row.id)
        except KeyError:
            all_shingles_test2[shingle] = shingle_id
            shingle_id += 1
            shingles_list_test.append([row.id])
    i += 1

i: 100000
i: 200000
i: 300000
i: 400000
i: 500000
i: 600000
i: 700000
i: 800000
i: 900000
i: 1000000
i: 1100000
i: 1200000
i: 1300000
i: 1400000
i: 1500000
i: 1600000
i: 1700000
i: 1800000
i: 1900000
i: 2000000
i: 2100000
i: 2200000
i: 2300000
i: 2400000
i: 2500000
i: 2600000
i: 2700000
i: 2800000
i: 2900000
i: 3000000
i: 3100000
i: 3200000
i: 3300000
i: 3400000
i: 3500000
i: 3600000
i: 3700000
i: 3800000
i: 3900000
i: 4000000
i: 4100000
i: 4200000
i: 4300000
i: 4400000
i: 4500000


In [115]:
shingles_list_test[0]

[1512186166438637582,
 1512189124714762241,
 1512196636533002250,
 1512225407189233664,
 1512248441350471680,
 1512250348135608327,
 1512253365618626566,
 1512255960747675651,
 1512124391408214019,
 1512164011009323016,
 1512171598819823618,
 1512091610573549573,
 1512053354272333825,
 1512410970651447298,
 1512370715105714179,
 1512371478771105796,
 1512379163193319425,
 1512382725461749760,
 1512385939401809925,
 1512387582805479424,
 1512403187247943684,
 1512297700624117766,
 1512307976723345412,
 1512276463956869123,
 1512229865587625984,
 1512113607038976000,
 1512030658444603392,
 1512595576046723072,
 1512448704950648834,
 1512762942789357572,
 1512466061110566914,
 1512569412188000259,
 1512439550718664713,
 1512511248235053059,
 1512615208107184132,
 1512659149452197897,
 1512546752858972162,
 1512651999719829507,
 1512595982537560064,
 1512582176352653313,
 1512606666545504257,
 1512506342010863616,
 1512462848097611780,
 1512605739625312256,
 1512720585192722437,
 151241312

In [110]:
reduced_data[reduced_data['id'] == 1512166838502785024].text.values

array(['RT @tuluslotrec: @BenitoBaranda @uchileradio Tuve ocasión de leerlos y estudiarlos en detalle, especialmente aquellos de las reivindicacion…'],
      dtype=object)

In [74]:
all_shingles_test2

{' años': 0,
 'reivi': 1,
 'ación': 2,
 'tegra': 3,
 ', la ': 4,
 'imas ': 5,
 ' hará': 6,
 'rá la': 7,
 'ará c': 8,
 'tituc': 9,
 ' d la': 10,
 'araci': 11,
 'l gol': 12,
 ' la m': 13,
 'indic': 14,
 'dicar': 15,
 'eivin': 16,
 'ntegr': 17,
 'a rep': 18,
 'ión i': 19,
 'argo ': 20,
 'l d l': 21,
 'rá ca': 22,
 'egral': 23,
 'ral d': 24,
 'CC: T': 25,
 'e, la': 26,
 'hará ': 27,
 'y rei': 28,
 'ctima': 29,
 'al d ': 30,
 'e har': 31,
 'RT @V': 32,
 ' del ': 33,
 'asi 5': 34,
 ' repa': 35,
 'íctim': 36,
 'ras c': 37,
 'si 50': 38,
 'lpe, ': 39,
 '@Vale': 40,
 'ón se': 41,
 'tució': 42,
 'pe, l': 43,
 'cargo': 44,
 'ituci': 45,
 'olpe,': 46,
 'leMir': 47,
 'stitu': 48,
 'ución': 49,
 'nstit': 50,
 'del g': 51,
 'randa': 52,
 'repar': 53,
 'se ha': 54,
 ' y re': 55,
 'ción ': 56,
 'o d l': 57,
 'vícti': 58,
 'cará ': 59,
 'parac': 60,
 's víc': 61,
 'la re': 62,
 'Miran': 63,
 ': Tra': 64,
 's y r': 65,
 'el go': 66,
 'C: Tr': 67,
 'as y ': 68,
 'Tras ': 69,
 ' @Val': 70,
 ' 50 a': 71,
 '

In [117]:
print(f"All Shingles test2: {total_size(all_shingles_test2)}")
print(f"Shingles list test: {total_size(shingles_list_test)}")

All Shingles test2: 530755830
Shingles list test: 4594963240


| Var | Tamaño |
|-----|--------:|
|all_shingles_test | 4974079270|
|all_shingles_set | 381308486|
|all_shingles_test2 | 530755830|
|shingles_list_test| 4594963240 |
